<a href="https://colab.research.google.com/github/umutcankarakas/lstm-musicgenerator/blob/master/convert2midi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
from pathlib import *
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord, stream
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
import string

In [2]:
!git clone https://github.com/umutcankarakas/lstm-musicgenerator.git
os.chdir('lstm-musicgenerator')

Cloning into 'lstm-musicgenerator'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 49 (delta 14), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (49/49), done.


In [3]:
train_path = 'data/'

In [4]:
def convertMidi2Arr(midi_element):
  newlist = []
  max_duration = 8*4*4
  if isinstance(midi_element, note.Note):
    newlist.append(midi_element.pitch.midi)
    if int(round(midi_element.duration.quarterLength*4)) > max_duration:
      newlist.append(128 + max_duration)
    else:
      newlist.append(128 + int(round(midi_element.duration.quarterLength*4)))
  elif isinstance(midi_element, chord.Chord):
    for n in midi_element:
      newlist.append(n.pitch.midi) 
      if int(round(midi_element.duration.quarterLength*4)) > max_duration:
        newlist.append(128 + max_duration)
      else:
        newlist.append(128 + int(round(midi_element.duration.quarterLength*4)))
  elif isinstance(midi_element, note.Rest):
    newlist.append(128)
    if int(round(midi_element.duration.quarterLength*4)) > max_duration:
      newlist.append(128 + max_duration)
    else:
      newlist.append(128 + int(round(midi_element.duration.quarterLength*4)))
  return newlist

In [5]:
#BUUU
def get_notes_old():
  mylist = []
  count = 0
  last_offset = 0.0
  last_only_chord = 0
  for file in glob.glob(train_path + '**/*.midi', recursive=True):
    try:
      midi = converter.parse(file)
    except:
      continue
    print("Parsing %s" % file)
    #print(count)
    notes_to_parse = None
    max_duration = 8*4*4

    try: # file has instrument parts
      s2 = instrument.partitionByInstrument(midi_piece)
      notes_to_parse = s2.parts[0].recurse() 
    except: # file has notes in a flat structure
      notes_to_parse = midi.flat.notesAndRests

    same_offset_elements = []
    min_duration = max_duration
    for element in notes_to_parse:
      print(element, element.duration, element.offset)
      if isinstance(element, note.Note) or isinstance(element, chord.Chord) or isinstance(element, note.Rest):
        if( not same_offset_elements or same_offset_elements[0].offset == element.offset):
          same_offset_elements.append(element)
          if(element.duration.quarterLength*4 < min_duration):
            min_duration = element.duration.quarterLength*4
        else:
          add_arr = []
          flags = [0,0,0]
          for toadd_midi in same_offset_elements:
            if isinstance(element, note.Note): flags[0] = 1
            elif isinstance(element, chord.Chord): flags[1] = 1
            elif isinstance(element, note.Rest): flags[2] = 1
            toadd = convertMidi2Arr(toadd_midi)
            if toadd[0]==128:
              add_arr.extend(toadd)
            add_arr.extend(toadd)
          
          if last_only_chord and mylist and mylist[-1] !='_':
            last_only_chord = 0
            temp = mylist[-1]
            mylist[-1] = 128 + int(round((same_offset_elements[0].offset - last_offset)*4))
            
            temp2 = mylist[-1]
            mylist.extend(add_arr)
            if(mylist and mylist[-1] != -1):
              mylist.append(257)
              mylist.append(128 + temp -temp2)
          
          else:
            mylist.extend(add_arr)
            #flagleri sifirla
            if(mylist and mylist[-1] != -1):
              mylist.append(257)
              mylist.append(128+int(round(min_duration)))

          if flags[0] == 0 and flags[1] == 1 and flags[2] == 0:
            last_only_chord = 0

          flags=[0,0,0]
          last_offset =  same_offset_elements[0].offset
          same_offset_elements = []
          min_duration = max_duration
          same_offset_elements.append(element)
          if(element.duration.quarterLength*4 < min_duration):
            min_duration = element.duration.quarterLength*4
        

          
    if count != 0: count = count + 1
    else: break
    mylist.append('_')

  with open('data/notes', 'wb') as filepath:
    pickle.dump(mylist, filepath)
  return mylist

In [6]:
def get_notes():
  mylist = []
  count = 0
  last_offset = 0.0
  offset_diff = 0.0
  for file in glob.glob(train_path + '**/*.midi', recursive=True):
    try:
      midi = converter.parse(file)
    except:
      continue
    print("Parsing %s" % file)
    #print(count)
    notes_to_parse = None
    max_duration = 8*4*4

    try: # file has instrument parts
      s2 = instrument.partitionByInstrument(midi_piece)
      notes_to_parse = s2.parts[0].recurse() 
    except: # file has notes in a flat structure
      notes_to_parse = midi.flat.notesAndRests

    same_offset_elements = []
    #min_duration = max_duration
    for element in notes_to_parse:
      print(element, element.duration, element.offset)
      if isinstance(element, note.Note) or isinstance(element, chord.Chord) or isinstance(element, note.Rest):
        if( not same_offset_elements or same_offset_elements[0].offset == element.offset):
          same_offset_elements.append(element)
        else:
          add_arr = []
          #flags = [0,0,0]
          for toadd_midi in same_offset_elements:
            toadd = convertMidi2Arr(toadd_midi)
            if toadd[0]==128:
              continue
            add_arr.extend(toadd)
          if(element.offset > last_offset):
            offset_diff = same_offset_elements[0].offset - last_offset
            last_offset = same_offset_elements[0].offset
          if mylist and mylist[-1] != '_':
            mylist.append(257)
            mylist.append(128+int(round(offset_diff*4)))
            mylist.extend(add_arr)
          else:
            mylist.extend(add_arr)

          offset_diff = 0.0
          same_offset_elements = []
          same_offset_elements.append(element)

    if same_offset_elements:
      add_arr = []
      for toadd_midi in same_offset_elements:
        toadd = convertMidi2Arr(toadd_midi)
        if toadd[0]==128:
          continue
        add_arr.extend(toadd)
      if(element.offset > last_offset):
        offset_diff = same_offset_elements[0].offset - last_offset
        last_offset = same_offset_elements[0].offset
      if mylist and mylist[-1] != '_':
        mylist.append(257)
        mylist.append(128+int(round(offset_diff*4)))
        mylist.extend(add_arr)
      else:
        mylist.extend(add_arr)

    if count != 0: count = count + 1
    else: break
    mylist.append('_')

  with open('data/notes', 'wb') as filepath:
    pickle.dump(mylist, filepath)
  return mylist

In [7]:
def create_midi(prediction_output):
  output_notes = []
  same_offset_elements = []
  same_offset_duration_list = []
  offset = 0.0
  prediction_output.append(257)
  prediction_output.append(128)
  if prediction_output[0] > 128 : prediction_output = prediction_output[1:]
  if prediction_output[0] == 257: prediction_output = prediction_output[2:]
  for pitch,duration in zip(prediction_output[0::2], prediction_output[1::2]):
    print(pitch,duration)
    if pitch!=257 and pitch != 128:
      new_note = note.Note(pitch)
      new_note.duration.quarterLength = (duration-128)/4
      new_note.storedInstrument = instrument.Piano()
      new_note.offset = offset
      same_offset_elements.append(new_note)
      same_offset_duration_list.append((duration-128)/4)
    elif(pitch == 128):
      new_note = note.Rest()
      new_note.duration.quarterLength = (duration-128)/4
      new_note.storedInstrument = instrument.Piano()
      new_note.offset = offset
      output_notes.append(new_note)
      continue
    else:
      if not same_offset_elements:
        same_offset_elements = []
        same_offset_duration_list = []
        offset = offset + (duration-128)/4
      elif len(same_offset_elements) == 1:
        offset = offset + (duration-128)/4
        output_notes.append(same_offset_elements[0])
        same_offset_elements = []
        same_offset_duration_list = []
      else:
        duration_set = set(same_offset_duration_list)
        chord_notes = []
        for dur in duration_set:
          for note_samp in same_offset_elements:
            if note_samp.duration.quarterLength == dur:
              chord_notes.append(note_samp)
          if len(chord_notes) == 1:
            output_notes.append(chord_notes[0])
            chord_notes = []
          else:
            new_chord = chord.Chord(chord_notes)
            new_chord.offset = offset
            new_chord.duration.quarterLength = dur
            output_notes.append(new_chord)
            chord_notes = []
        offset = offset + (duration-128)/4
        same_offset_elements = []
        same_offset_duration_list = []

  midi_stream = stream.Stream(output_notes)
  midi_stream.write('midi', fp='test_output.mid')
  return output_notes


In [8]:
midi = get_notes()


Parsing data/AmericanBeautyRag.midi
<music21.note.Rest rest> <music21.duration.Duration 1.75> 0.0
<music21.note.Rest rest> <music21.duration.Duration 2.25> 0.0
<music21.note.Rest rest> <music21.duration.Duration 1.5> 0.0
<music21.note.Rest rest> <music21.duration.Duration 5.0> 0.0
<music21.note.Rest rest> <music21.duration.Duration 297.0> 0.0
<music21.chord.Chord B-1 B-2> <music21.duration.Duration 1/3> 1.5
<music21.note.Note B-> <music21.duration.Duration 0.25> 1.75
<music21.note.Note E-> <music21.duration.Duration 1/3> 2.0
<music21.chord.Chord E-3 E-2> <music21.duration.Duration 0.5> 2.0
<music21.note.Note F> <music21.duration.Duration 1/3> 2.25
<music21.note.Rest rest> <music21.duration.Duration 291.75> 2.25
<music21.note.Note G> <music21.duration.Duration 1/3> 2.5
<music21.chord.Chord G4 E-4 B-3> <music21.duration.Duration 0.25> 2.5
<music21.note.Note B-> <music21.duration.Duration 1/3> 2.75
<music21.chord.Chord E-2 E-3> <music21.duration.Duration 0.5> 3.0
<music21.chord.Chord G#5 

In [9]:
midi

[34,
 129,
 46,
 129,
 257,
 129,
 70,
 129,
 257,
 129,
 75,
 129,
 51,
 130,
 39,
 130,
 257,
 129,
 77,
 129,
 257,
 129,
 79,
 129,
 67,
 129,
 63,
 129,
 58,
 129,
 257,
 129,
 82,
 129,
 257,
 129,
 39,
 130,
 51,
 130,
 257,
 129,
 80,
 131,
 72,
 131,
 257,
 129,
 63,
 129,
 60,
 129,
 68,
 129,
 257,
 129,
 87,
 129,
 257,
 129,
 51,
 130,
 39,
 130,
 257,
 129,
 70,
 131,
 79,
 131,
 257,
 129,
 63,
 129,
 67,
 129,
 58,
 129,
 257,
 129,
 87,
 129,
 257,
 129,
 78,
 129,
 71,
 130,
 63,
 131,
 66,
 132,
 57,
 132,
 257,
 129,
 83,
 129,
 257,
 130,
 79,
 129,
 70,
 129,
 82,
 129,
 257,
 129,
 63,
 135,
 67,
 135,
 58,
 135,
 257,
 129,
 84,
 129,
 257,
 129,
 82,
 130,
 257,
 129,
 79,
 129,
 257,
 129,
 75,
 130,
 257,
 129,
 72,
 129,
 257,
 129,
 70,
 130,
 257,
 129,
 67,
 129,
 257,
 129,
 63,
 130,
 257,
 129,
 65,
 129,
 257,
 129,
 66,
 130,
 257,
 129,
 67,
 129,
 257,
 129,
 43,
 130,
 55,
 130,
 257,
 129,
 74,
 129,
 257,
 129,
 63,
 129,
 69,
 129,
 72,
 129,
 

In [10]:
out = create_midi(midi)

34 129
46 129
257 129
70 129
257 129
75 129
51 130
39 130
257 129
77 129
257 129
79 129
67 129
63 129
58 129
257 129
82 129
257 129
39 130
51 130
257 129
80 131
72 131
257 129
63 129
60 129
68 129
257 129
87 129
257 129
51 130
39 130
257 129
70 131
79 131
257 129
63 129
67 129
58 129
257 129
87 129
257 129
78 129
71 130
63 131
66 132
57 132
257 129
83 129
257 130
79 129
70 129
82 129
257 129
63 135
67 135
58 135
257 129
84 129
257 129
82 130
257 129
79 129
257 129
75 130
257 129
72 129
257 129
70 130
257 129
67 129
257 129
63 130
257 129
65 129
257 129
66 130
257 129
67 129
257 129
43 130
55 130
257 129
74 129
257 129
63 129
69 129
72 129
54 129
42 129
257 130
68 133
62 133
70 133
41 130
53 130
257 130
56 129
58 129
65 129
257 130
34 130
46 130
257 129
69 129
257 129
70 129
56 129
58 129
62 129
257 129
71 129
257 129
72 129
53 130
41 130
257 129
73 129
257 129
56 129
58 129
62 129
257 129
68 129
74 129
257 129
34 130
46 130
257 129
79 129
257 129
77 129
68 129
74 129
56 129
58 129
62 1

In [11]:
for i in out:
  print(i, i.duration,i.offset)

<music21.chord.Chord B-1 B-2> <music21.duration.Duration 0.25> 0.0
<music21.note.Note B-> <music21.duration.Duration 0.25> 0.25
<music21.note.Note E-> <music21.duration.Duration 0.25> 0.5
<music21.chord.Chord E-3 E-2> <music21.duration.Duration 0.5> 0.5
<music21.note.Note F> <music21.duration.Duration 0.25> 0.75
<music21.chord.Chord G5 G4 E-4 B-3> <music21.duration.Duration 0.25> 1.0
<music21.note.Note B-> <music21.duration.Duration 0.25> 1.25
<music21.chord.Chord E-2 E-3> <music21.duration.Duration 0.5> 1.5
<music21.chord.Chord G#5 C5> <music21.duration.Duration 0.75> 1.75
<music21.chord.Chord E-4 C4 G#4> <music21.duration.Duration 0.25> 2.0
<music21.note.Note E-> <music21.duration.Duration 0.25> 2.25
<music21.chord.Chord E-3 E-2> <music21.duration.Duration 0.5> 2.5
<music21.chord.Chord B-4 G5> <music21.duration.Duration 0.75> 2.75
<music21.chord.Chord E-4 G4 B-3> <music21.duration.Duration 0.25> 3.0
<music21.note.Note E-> <music21.duration.Duration 0.25> 3.25
<music21.note.Note F#> <